# Timesheet Overlap Calculation

Efficient overlap detection for timesheets using **interval sorting + sweep** approach.

**Fields:** `name`, `date_time` (start), `date_time_end` (end), `user_id`

**Algorithm:** For each `user_id`, sort intervals by start time, then check adjacent intervals — `O(n log n)` per user.

In [3]:
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

## 1. Core Overlap Detection Engine

In [4]:
def find_overlaps(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find all overlapping timesheet entries efficiently.
    
    Algorithm:
    1. Group by user_id (overlaps only matter within the same user)
    2. Sort each group by date_time (start)
    3. Sweep through sorted intervals — if current start < previous end, overlap detected
    
    Time complexity: O(n log n) per user due to sorting
    Space complexity: O(k) where k = number of overlaps found
    
    Parameters:
        df: DataFrame with columns [name, date_time, date_time_end, user_id]
    
    Returns:
        DataFrame of overlapping pairs with details
    """
    # Validate input
    required_cols = {'name', 'date_time', 'date_time_end', 'user_id'}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")
    
    # Ensure datetime types
    df = df.copy()
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['date_time_end'] = pd.to_datetime(df['date_time_end'])
    
    # Validate: start must be before end
    invalid = df[df['date_time'] >= df['date_time_end']]
    if len(invalid) > 0:
        print(f"⚠️ Warning: {len(invalid)} entries have start >= end. These will be skipped.")
        df = df[df['date_time'] < df['date_time_end']]
    
    overlaps = []
    
    # Group by user — overlaps only matter per user
    for user_id, group in df.groupby('user_id'):
        # Sort by start time within each user group
        group = group.sort_values('date_time').reset_index(drop=True)
        
        # Track the "max end time seen so far" and corresponding entry
        # This handles cases where a short entry is fully contained in a longer one
        max_end_idx = 0
        
        for i in range(1, len(group)):
            # Compare current entry's start with the max end time seen so far
            if group.loc[i, 'date_time'] < group.loc[max_end_idx, 'date_time_end']:
                overlaps.append({
                    'user_id': user_id,
                    'entry_a_name': group.loc[max_end_idx, 'name'],
                    'entry_a_start': group.loc[max_end_idx, 'date_time'],
                    'entry_a_end': group.loc[max_end_idx, 'date_time_end'],
                    'entry_b_name': group.loc[i, 'name'],
                    'entry_b_start': group.loc[i, 'date_time'],
                    'entry_b_end': group.loc[i, 'date_time_end'],
                    'overlap_start': group.loc[i, 'date_time'],
                    'overlap_end': min(group.loc[i, 'date_time_end'], group.loc[max_end_idx, 'date_time_end']),
                })
            
            # Also check overlap with the immediately previous entry (if different from max_end)
            if max_end_idx != i - 1:
                if group.loc[i, 'date_time'] < group.loc[i - 1, 'date_time_end']:
                    overlaps.append({
                        'user_id': user_id,
                        'entry_a_name': group.loc[i - 1, 'name'],
                        'entry_a_start': group.loc[i - 1, 'date_time'],
                        'entry_a_end': group.loc[i - 1, 'date_time_end'],
                        'entry_b_name': group.loc[i, 'name'],
                        'entry_b_start': group.loc[i, 'date_time'],
                        'entry_b_end': group.loc[i, 'date_time_end'],
                        'overlap_start': group.loc[i, 'date_time'],
                        'overlap_end': min(group.loc[i, 'date_time_end'], group.loc[i - 1, 'date_time_end']),
                    })
            
            # Update max_end tracker
            if group.loc[i, 'date_time_end'] > group.loc[max_end_idx, 'date_time_end']:
                max_end_idx = i
    
    if not overlaps:
        return pd.DataFrame(columns=[
            'user_id', 'entry_a_name', 'entry_a_start', 'entry_a_end',
            'entry_b_name', 'entry_b_start', 'entry_b_end',
            'overlap_start', 'overlap_end'
        ])
    
    result = pd.DataFrame(overlaps)
    result['overlap_duration'] = result['overlap_end'] - result['overlap_start']
    
    # Deduplicate (in case both checks caught the same pair)
    result = result.drop_duplicates(
        subset=['user_id', 'entry_a_start', 'entry_a_end', 'entry_b_start', 'entry_b_end']
    ).reset_index(drop=True)
    
    return result

In [5]:
def validate_new_entry(
    existing_df: pd.DataFrame,
    name: str,
    date_time: str,
    date_time_end: str,
    user_id: str
) -> Tuple[bool, Optional[pd.DataFrame]]:
    """
    Check if a new timesheet entry would overlap with existing entries.
    
    Uses binary search on sorted intervals for O(log n) lookup per user.
    
    Parameters:
        existing_df: Current timesheet DataFrame
        name: Name/description of the new entry
        date_time: Start datetime (string or datetime)
        date_time_end: End datetime (string or datetime)
        user_id: User identifier
    
    Returns:
        Tuple of (is_valid: bool, conflicting_entries: DataFrame or None)
    """
    new_start = pd.to_datetime(date_time)
    new_end = pd.to_datetime(date_time_end)
    
    if new_start >= new_end:
        raise ValueError("Start time must be before end time")
    
    if existing_df.empty:
        return True, None
    
    df = existing_df.copy()
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['date_time_end'] = pd.to_datetime(df['date_time_end'])
    
    # Filter to same user only
    user_entries = df[df['user_id'] == user_id].copy()
    
    if user_entries.empty:
        return True, None
    
    # Two intervals [A_start, A_end) and [B_start, B_end) overlap iff:
    #   A_start < B_end AND B_start < A_end
    conflicts = user_entries[
        (user_entries['date_time'] < new_end) & 
        (new_start < user_entries['date_time_end'])
    ]
    
    if conflicts.empty:
        return True, None
    else:
        return False, conflicts[['name', 'date_time', 'date_time_end', 'user_id']]

## 2. Sample Data & Demo

In [6]:
# Create sample timesheet data with intentional overlaps
sample_data = [
    # User A — normal entries (no overlap)
    {'name': 'Morning standup',    'date_time': '2026-02-26 09:00', 'date_time_end': '2026-02-26 09:30', 'user_id': 'user_a'},
    {'name': 'Feature dev',        'date_time': '2026-02-26 09:30', 'date_time_end': '2026-02-26 12:00', 'user_id': 'user_a'},
    {'name': 'Lunch break',        'date_time': '2026-02-26 12:00', 'date_time_end': '2026-02-26 13:00', 'user_id': 'user_a'},
    {'name': 'Code review',        'date_time': '2026-02-26 13:00', 'date_time_end': '2026-02-26 14:30', 'user_id': 'user_a'},
    
    # User A — OVERLAPPING entries
    {'name': 'Client call',        'date_time': '2026-02-26 14:00', 'date_time_end': '2026-02-26 15:00', 'user_id': 'user_a'},  # overlaps with 'Code review'
    {'name': 'Bug fix',            'date_time': '2026-02-26 14:45', 'date_time_end': '2026-02-26 16:00', 'user_id': 'user_a'},  # overlaps with 'Client call'
    
    # User B — contains a fully nested entry
    {'name': 'Project planning',   'date_time': '2026-02-26 08:00', 'date_time_end': '2026-02-26 12:00', 'user_id': 'user_b'},
    {'name': 'Quick sync',         'date_time': '2026-02-26 10:00', 'date_time_end': '2026-02-26 10:30', 'user_id': 'user_b'},  # fully inside 'Project planning'
    {'name': 'Documentation',      'date_time': '2026-02-26 13:00', 'date_time_end': '2026-02-26 15:00', 'user_id': 'user_b'},
    
    # User C — no overlaps at all
    {'name': 'Design review',      'date_time': '2026-02-26 09:00', 'date_time_end': '2026-02-26 10:00', 'user_id': 'user_c'},
    {'name': 'Sprint planning',    'date_time': '2026-02-26 10:00', 'date_time_end': '2026-02-26 11:30', 'user_id': 'user_c'},
    {'name': 'Testing',            'date_time': '2026-02-26 13:00', 'date_time_end': '2026-02-26 15:00', 'user_id': 'user_c'},
]

df = pd.DataFrame(sample_data)
print(f"Total entries: {len(df)}")
print(f"Users: {df['user_id'].nunique()}")
df

Total entries: 12
Users: 3


,name,date_time,date_time_end,user_id
0,Morning standup,2026-02-26 09:00,2026-02-26 09:30,user_a
1,Feature dev,2026-02-26 09:30,2026-02-26 12:00,user_a
2,Lunch break,2026-02-26 12:00,2026-02-26 13:00,user_a
3,Code review,2026-02-26 13:00,2026-02-26 14:30,user_a
4,Client call,2026-02-26 14:00,2026-02-26 15:00,user_a
5,Bug fix,2026-02-26 14:45,2026-02-26 16:00,user_a
6,Project planning,2026-02-26 08:00,2026-02-26 12:00,user_b
7,Quick sync,2026-02-26 10:00,2026-02-26 10:30,user_b
8,Documentation,2026-02-26 13:00,2026-02-26 15:00,user_b
9,Design review,2026-02-26 09:00,2026-02-26 10:00,user_c


## 3. Detect All Overlaps

In [7]:
overlaps = find_overlaps(df)

if overlaps.empty:
    print("✅ No overlapping entries found!")
else:
    print(f"❌ Found {len(overlaps)} overlap(s):\n")
    for idx, row in overlaps.iterrows():
        print(f"  [{row['user_id']}] \"{row['entry_a_name']}\" ({row['entry_a_start'].strftime('%H:%M')}-{row['entry_a_end'].strftime('%H:%M')})")
        print(f"       ↔ \"{row['entry_b_name']}\" ({row['entry_b_start'].strftime('%H:%M')}-{row['entry_b_end'].strftime('%H:%M')})")
        print(f"       Overlap: {row['overlap_start'].strftime('%H:%M')}-{row['overlap_end'].strftime('%H:%M')} ({row['overlap_duration']})")
        print()

overlaps

❌ Found 3 overlap(s):

  [user_a] "Code review" (13:00-14:30)
       ↔ "Client call" (14:00-15:00)
       Overlap: 14:00-14:30 (0 days 00:30:00)

  [user_a] "Client call" (14:00-15:00)
       ↔ "Bug fix" (14:45-16:00)
       Overlap: 14:45-15:00 (0 days 00:15:00)

  [user_b] "Project planning" (08:00-12:00)
       ↔ "Quick sync" (10:00-10:30)
       Overlap: 10:00-10:30 (0 days 00:30:00)



,user_id,entry_a_name,entry_a_start,entry_a_end,entry_b_name,entry_b_start,entry_b_end,overlap_start,overlap_end,overlap_duration
0,user_a,Code review,2026-02-26 13:00:00,2026-02-26 14:30:00,Client call,2026-02-26 14:00:00,2026-02-26 15:00:00,2026-02-26 14:00:00,2026-02-26 14:30:00,0 days 00:30:00
1,user_a,Client call,2026-02-26 14:00:00,2026-02-26 15:00:00,Bug fix,2026-02-26 14:45:00,2026-02-26 16:00:00,2026-02-26 14:45:00,2026-02-26 15:00:00,0 days 00:15:00
2,user_b,Project planning,2026-02-26 08:00:00,2026-02-26 12:00:00,Quick sync,2026-02-26 10:00:00,2026-02-26 10:30:00,2026-02-26 10:00:00,2026-02-26 10:30:00,0 days 00:30:00


## 4. Validate a New Entry Before Insertion

In [8]:
# Example: Try adding a new entry that WILL conflict
print("=" * 60)
print("TEST 1: Entry that conflicts with existing timesheet")
print("=" * 60)

is_valid, conflicts = validate_new_entry(
    existing_df=df,
    name='New meeting',
    date_time='2026-02-26 09:15',
    date_time_end='2026-02-26 10:00',
    user_id='user_a'
)

if is_valid:
    print("✅ Entry is valid — no overlaps.")
else:
    print(f"❌ Entry conflicts with {len(conflicts)} existing entry/entries:")
    display(conflicts)

print()
print("=" * 60)
print("TEST 2: Entry that does NOT conflict")
print("=" * 60)

is_valid, conflicts = validate_new_entry(
    existing_df=df,
    name='Evening work',
    date_time='2026-02-26 17:00',
    date_time_end='2026-02-26 18:00',
    user_id='user_a'
)

if is_valid:
    print("✅ Entry is valid — no overlaps.")
else:
    print(f"❌ Entry conflicts with {len(conflicts)} existing entry/entries:")
    display(conflicts)

TEST 1: Entry that conflicts with existing timesheet
❌ Entry conflicts with 2 existing entry/entries:


,name,date_time,date_time_end,user_id
0,Morning standup,2026-02-26 09:00:00,2026-02-26 09:30:00,user_a
1,Feature dev,2026-02-26 09:30:00,2026-02-26 12:00:00,user_a



TEST 2: Entry that does NOT conflict
✅ Entry is valid — no overlaps.


## 5. Summary Report by User

In [9]:
def overlap_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate a per-user summary: total entries, overlaps found, total overlap duration.
    """
    overlaps = find_overlaps(df)
    
    users = df['user_id'].unique()
    summary = []
    
    for user_id in sorted(users):
        user_entries = df[df['user_id'] == user_id]
        user_overlaps = overlaps[overlaps['user_id'] == user_id] if not overlaps.empty else pd.DataFrame()
        
        total_duration = (
            pd.to_datetime(user_entries['date_time_end']) - pd.to_datetime(user_entries['date_time'])
        ).sum()
        
        overlap_duration = user_overlaps['overlap_duration'].sum() if not user_overlaps.empty else timedelta(0)
        
        summary.append({
            'user_id': user_id,
            'total_entries': len(user_entries),
            'overlapping_pairs': len(user_overlaps),
            'total_logged_time': str(total_duration),
            'total_overlap_time': str(overlap_duration),
            'status': '❌ Has overlaps' if len(user_overlaps) > 0 else '✅ Clean'
        })
    
    return pd.DataFrame(summary)

summary = overlap_summary(df)
summary

,user_id,total_entries,overlapping_pairs,total_logged_time,total_overlap_time,status
0,user_a,6,2,0 days 07:45:00,0 days 00:45:00,❌ Has overlaps
1,user_b,3,1,0 days 06:30:00,0 days 00:30:00,❌ Has overlaps
2,user_c,3,0,0 days 04:30:00,0:00:00,✅ Clean


## 6. Performance Notes

| Operation | Complexity | Description |
|---|---|---|
| `find_overlaps()` | **O(n log n)** per user | Sort + single sweep — optimal for batch detection |
| `validate_new_entry()` | **O(m)** per user | Filter user entries + vectorized comparison, where m = user's entries |
| Naive pairwise check | O(n²) | ❌ Avoided — would check every pair against every other |

### Why this is efficient:
- **Grouping by `user_id` first** — entries from different users can never overlap, so we eliminate cross-user comparisons entirely
- **Sort + sweep** — after sorting by start time, we only compare each entry with its predecessor (and track the max-end for nested intervals), instead of checking all pairs
- **Vectorized validation** — `validate_new_entry()` uses pandas boolean indexing instead of Python loops